In [86]:
import csv
import json
import pandas as pd
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer
from elasticsearch import Elasticsearch
import os
from groq import Groq

In [87]:
# Load the JSON file
with open('data/documents-with-ids.json', 'r') as f:
    documents = json.load(f)

df_ground_truth = pd.read_csv('data/ground-truth-data.csv')
ground_truth = df_ground_truth.to_dict(orient='records')

In [88]:
model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

/workspaces/Econet_Chatbot/venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [89]:
es_client = Elasticsearch('http://localhost:9200')

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "question": {"type": "text"},
            "answer": {"type": "text"},
            "id": {"type": "text"},
            "question_answer_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "customer-support"
es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'customer-support'})

In [90]:
for doc in tqdm(documents):
    question = doc['Question']
    answer = doc['Answer']
    
    es_doc = {
        "question": question,
        "answer": answer,
        "question_answer_vector": model.encode(question + ' ' + answer)
    }
    
    es_client.index(index=index_name, document=es_doc)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 194/194 [00:09<00:00, 19.51it/s]


In [91]:
def elastic_search_knn(field, vector):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000
    }
    search_query = {
        "knn": knn,
        "_source": ["question", "answer"]
    }
    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])
    return result_docs

In [98]:
def question_answer_vector_knn(q):
    if isinstance(q, dict):
        question = q.get('Question') or q.get('question')
    else:
        question = q
    if not question:
        raise ValueError("Input must be a string or a dictionary with a 'Question' or 'question' key")
    v_q = model.encode(question)
    return elastic_search_knn('question_answer_vector', v_q)

os.environ["GROQ_API_KEY"] = "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"

In [99]:
def build_prompt(query, search_results):
    prompt_template = """
You are a highly knowledgeable and helpful customer support assistant for a
telecommunications company. Your job is to assist customers by answering
their questions with accurate and clear information based on the CONTEXT provided.
Only use the facts from the CONTEXT when answering the customer's QUESTION.
Be concise, professional, and empathetic in your responses. 
If the CONTEXT does not fully answer the QUESTION, politely suggest that the 
customer reach out to a live support agent for further assistance.
QUESTION: {question}
CONTEXT: 
{context}
""".strip()
    context = ""
    
    for doc in search_results:
        context = context + f"question: {doc['question']}\nanswer: {doc['answer']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [100]:
def build_prompt(query, search_results):
    prompt_template = """
You are a highly knowledgeable, friendly, and empathetic  and helpful customer support assistant for a telecommunications company.
Your role is to assist customers by answering their questions with accurate, clear, and concise information based on the CONTEXT provided. 
Please respond in a conversational tone that makes the customer feel heard and understood and Be concise, professional, and empathetic in your responses. 
Use only the facts from the CONTEXT when answering the customer's QUESTION.
If the CONTEXT does not have the   context to  answer the QUESTION, gently suggest that the customer 
reach out to a live support agent for further assistance.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()
    
    context = ""
    
    for doc in search_results:
        context += f"question: {doc['question']}\nanswer: {doc['answer']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt


In [107]:
def llm(prompt, model='llama-3.2-90b-text-p'):
    client = Groq()
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

def rag(query, model='llama-3.1-70b-versatile') -> str:
    if isinstance(query, dict):
        question = query.get('Question') or query.get('question')
    else:
        question = query
    if not question:
        raise ValueError("Input must be a string or a dictionary with a 'Question' or 'question' key")
    
    search_results = question_answer_vector_knn(question)
    prompt = build_prompt(question, search_results)
    answer = llm(prompt, model=model)
    return answer

In [108]:
ground_truth[20]

{'Question': "I am trying to borrow money from my airtime account, but I'm not allowed, Why is this?",
 'Answer': 'Please note, in order to borrow airtime, your line has to be active for 3 months after the day of purchase. you also need to recharge more often to be eligible for credit.',
 'id': '53986b1a'}

In [109]:
# Example usage
rag(ground_truth[88])

"I understand your question, and I'd be happy to help. Based on the information available, I can confirm that the bundle minutes can be used to call an Econet mobile number that is roaming outside the country. However, I want to clarify that this only applies if the bundle is being used for calls from within Zimbabwe. If you have any other questions or need further clarification, please feel free to ask, and I'll do my best to assist you."

### Cosine similarity metric

In [65]:
answer_orig = 'Can the bundle minutes be used outside the country for Econet mobile numbers'
answer_llm = 'The bundles are only valid for local Econet-to-Econet calls. However, the bundle can be used if calling a number that is roaming outside the country.'

In [66]:
v_llm = model.encode(answer_llm)
v_orig = model.encode(answer_orig)

v_llm.dot(v_orig)

0.7694156